In [4]:
from jax_tools.human import *
H = 30 # num of steps in MPC horizon

hp = HumanParams()
rp = RewardParams()
mp = MPCParams(goal=np.array([0.3, *[0]*(nx-1)]))

hum = Human(hp, rp, H=H)
#print(hum.solve(mp)['u'].tau)
%timeit hum.solve(mp)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

12.7 ms ± 611 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
from casadi_tools.human import *

hp = HumanParams()
rp = RewardParams()
mp = MPCParams(goal=np.array([0.3, *[0]*(nx-1)]))
#print(hum.solve(mp).tau)
%timeit hum.solve(mp)

10 ms ± 83.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
# multiple shooting by hand
from jax.flatten_util import ravel_pytree
from jax_tools.human import *
from jax_tools.ipopt import nlpsol

H = 60

u = Human.Input(tau=np.zeros((H, nq)))
x = Human.State(mu=np.zeros((H,2*nq)), cov=np.zeros((H,2*nq)))

step_vmap = jax.vmap(hum._step, in_axes=(0, 0, None))
cost_vmap = jax.vmap(hum._cost, in_axes=(Human.Input(0), Human.State(0,0), None, None))

inp = {'x':x, 'u':u}
x0, unraveller = ravel_pytree(inp)

def con_eq(dec_vec):
    obj_inp = unraveller(dec_vec)
    x_next = step_vmap(obj_inp['u'], obj_inp['x'], hp)
    g = []
    g += [np.reshape(x_next.mu[:-1,:]-obj_inp['x'].mu[1:,:],(-1,1))]
    g += [np.reshape(x_next.cov[:-1,:]-obj_inp['x'].cov[1:,:],(-1,1))]

    # Intial value constraints
    g += [np.reshape(np.zeros((1,2*nq))-obj_inp['x'].mu[0,:],(-1,1))]
    g += [np.reshape(np.zeros((1,2*nq))-obj_inp['x'].cov[0,:],(-1,1))]
    return jnp.squeeze(jnp.concatenate(tuple(g)))    

def obj(dec_vec):
    obj_inp = unraveller(dec_vec)
    cost = cost_vmap(obj_inp['u'], obj_inp['x'], mp, rp)
    return np.sum(cost['r'])+np.sum(cost['r_discount'])

solver = nlpsol(obj = obj, con_eq = con_eq, con_ineq=None, opts={'disp':5})
soln = solver(x0)
%timeit solver(x0)
print(unraveller(soln.x)['u'])

ValueError: matmul input operand 1 must have ndim at least 1, but it has ndim 0